In [37]:
import networkx as nx
import pickle
import random
import scipy
from scipy.sparse import tril, find
import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import igraph
from django.core.management import call_command

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.tasks import *
import tmv_app
import tmv_app.utils.plotting as tplots
from tmv_app.utils.plotting import SquareCollection, CoordSquare

pids = [148, 210, 147, 186]

In [35]:
RunStats.objects.filter(query=7360,K__gt=90).delete()

(0, {})

In [131]:
s = 'USAGE OF FORAGE PLANTS TO PREVENT EROSION'
#d = Doc.objects.get(tslug=Doc.make_tslug(s))
d = Doc.objects.get(pk=897719)
print(d.title)
print(d.wosarticle.de)
print(d.content)
print(d.wosarticle.kwp)

list(d.query.filter(project__in=pids).values('id','title','text'))

Health risks of warming of 1.5 degrees C, 2 degrees C, and higher, above pre-industrial temperatures
climate change; dengue; occupational health; malaria; temperature-related mortality; undernutrition; ozone
Background: In response to the Paris Agreement under the United Nations Framework Convention on Climate Change, the research community was asked to estimate differences in sectoral-specific risks at 1.5 degrees C and 2 degrees C increases in global mean surface air temperature (SAT) above pre-industrial temperatures. Projections of the health risks of climate change typically focus on time periods and not on the magnitude of temperature change. Objective: Summarize projections of health risks associated with temperature extremes and occupational heat stress, air quality, undernutrition, and vector-borne diseases to estimate how these risks would differ at increases in warming of 1.5 degrees C, 2 degrees C, and higher. Methods: A comprehensive search strategy included English langua

[{'id': 7893,
  'text': '(TS=(("CO2" OR "GHG" OR "greenhouse gas" OR "climate change" OR "global warming" OR “climate crisis”) AND (“non-mitigation policies” OR “welfare” OR “housing” or “employment” OR “urban transport” OR “air pollution”))) NOT WC = ( BIOPHYSICS OR OPTICS OR LIMNOLOGY OR PHYSIOLOGY OR VETERINARY SCIENCES OR COMPUTER SCIENCE INTERDISCIPLINARY APPLICATIONS OR MICROBIOLOGY OR NUTRITION DIETETICS OR ORNITHOLOGY OR ZOOLOGY OR PALEONTOLOGY OR MATERIALS SCIENCE MULTIDISCIPLINARY OR PARASITOLOGY OR EVOLUTIONARY BIOLOGY OR RESPIRATORY SYSTEM OR SOIL SCIENCE OR IMAGING SCIENCE PHOTOGRAPHIC TECHNOLOGY OR ELECTROCHEMISTRY OR ALLERGY OR CHEMISTRY PHYSICAL OR GEOCHEMISTRY GEOPHYSICS OR INSTRUMENTS INSTRUMENTATION OR TOXICOLOGY OR BIOCHEMISTRY MOLECULAR BIOLOGY OR IMMUNOLOGY OR COMPUTER SCIENCE THEORY METHODS OR ENGINEERING CHEMICAL OR COMPUTER SCIENCE ARTIFICIAL INTELLIGENCE OR BIOTECHNOLOGY APPLIED MICROBIOLOGY OR CHEMISTRY APPLIED OR CHEMISTRY ANALYTICAL OR INFECTIOUS DISEASES O

In [26]:
bad_top = Topic.objects.get(pk=192847)

print(bad_top.top_words)

thresh = 0.02

docs = Doc.objects.filter(doctopic__topic=bad_top, doctopic__score__gt=thresh)

d = docs.order_by('doctopic__score')[2000]
ts = Topic.objects.filter(
    doctopic__doc=d,run_id=bad_top.run_id,doctopic__score__gt=thresh
).order_by('-doctopic__score')
print(ts)
print(d.content)

docs.count()

['cell', 'culture', 'growth', 'rate', 'medium', 'acid', 'concentration', 'treatment', 'high', 'effect']
<QuerySet [<Topic: {cell, culture, growth}>]>
Short-term (24 h) experiments were performed to examine the effect of anthracene (ANT) on Chlamydomonas reinhardtii cw92 grown in a batch culture system aerated with 2.5% CO2 At concentrations ranging from 0.7 to 5.6 mu M, ANT inhibited the growth of population in a concentration-dependent manner and EC50 calculated amounted to 1.6 mu M. At concentrations from 0.7 to 4.2 mu M ANT stimulated respiration and inhibited the intensity of photosynthesis but did not affect chlorophyll content in the cells. ANT influenced chlorophyll a fluorescence parameters, measured by OJIP test (O, J, I and P are the different steps of fluorescence induction curve). ANT diminished the performance index (PI), the yield of primary photochemistry (phi(Po)), the yield of electron transport (phi(Eo,) the efficiency of moving the electron beyond Qa(-) (Psi(0)) and 

3034

In [6]:
docs.filter(query__database="WoS",query__project__in=pids).values('query__title','query__r_count').annotate(
    n = Count('pk')
).order_by('-n')

<QuerySet [{'query__title': 'Query E_renew', 'query__r_count': 4098, 'n': 2419}, {'query__title': 'Query K_renew', 'query__r_count': 5343, 'n': 723}, {'query__title': 'Query L_renew', 'query__r_count': 9760, 'n': 586}, {'query__title': 'A literature survey on demand-side solutions for climate-change mitigation', 'query__r_count': 5378, 'n': 566}, {'query__title': 'Quantifying the potential for climate change mitigation of consumption-based options', 'query__r_count': 5257, 'n': 456}, {'query__title': 'Status consciousness in energy consumption decisions: A systematic review of literature', 'query__r_count': 3237, 'n': 245}, {'query__title': 'Determinants of low-carbon transport mode adoption: Systematic review of reviews', 'query__r_count': 4638, 'n': 238}, {'query__title': 'Mitigating climate change in food consumption and food waste: A systematic map of behavioural interventions', 'query__r_count': 1045, 'n': 209}, {'query__title': 'Query N_renew', 'query__r_count': 4704, 'n': 163}, 

In [135]:
bad_topic_ids = [
    198423,
    198376,
    198396,
    198375,
    198403,
    198368,
    198371,
    198358,
    198355,
    198422,
    198393,
    198381,
    198417,
    198378,
    198395
]
bad_topics = Topic.objects.filter(pk__in=bad_topic_ids)
list(bad_topics.values('title'))

[{'title': '{growth, rate, population}'},
 {'title': '{region, regional, area}'},
 {'title': '{plant, leaf, root}'},
 {'title': '{ch4, methane, n2o}'},
 {'title': '{control, load, controller}'},
 {'title': '{model, simulation, prediction}'},
 {'title': '{system, performance, operation}'},
 {'title': '{climate, impact, global}'},
 {'title': '{temperature, degree, increase}'},
 {'title': '{intensity, factor, efficiency}'},
 {'title': '{data, use, information}'},
 {'title': '{cell, culture, medium}'},
 {'title': '{vulnerability, resilience, drought}'},
 {'title': '{specie, ecosystem, fish}'},
 {'title': '{design, method, approach}'}]

In [185]:
doc_ids = set([])
n_docs = 0
for t in bad_topics:
    print(t)
    docs = Doc.objects.filter(
        doctopic__topic=t,doctopic__score__gt=0.04
    )
    doc_ids = doc_ids | set(docs.values_list('pk',flat=True))
    print(docs.count())
    n_docs+=docs.count()
    
print(n_docs)
len(doc_ids)

{growth, rate, population}
534
{region, regional, area}
468
{plant, leaf, root}
1039
{ch4, methane, n2o}
718
{control, load, controller}
568
{model, simulation, prediction}
594
{system, performance, operation}
637
{climate, impact, global}
779
{temperature, degree, increase}
1055
{intensity, factor, efficiency}
671
{data, use, information}
277
{cell, culture, medium}
867
{vulnerability, resilience, drought}
719
{specie, ecosystem, fish}
1109
{design, method, approach}
279
10314


9806

In [186]:
other_topics = Topic.objects.filter(
    run_id=t.run_id
).exclude(pk__in=bad_topic_ids)

other_t_docs = Doc.objects.filter(
    pk__in=doc_ids, 
    doctopic__topic__in=other_topics,
    doctopic__score__gt=0.02
)
other_t_doc_ids = set(other_t_docs.values_list('pk',flat=True))
len(other_t_doc_ids)


7508

In [187]:
t.run_id

<RunStats: RunStats object (2624)>

In [188]:
list(Doc.objects.filter(
    pk__in=random.sample(other_t_doc_ids,1),
    doctopic__topic__run_id=t.run_id,
    doctopic__score__gt=0.008
).values(
    'title',
    'content',
    'id',
    'wosarticle__de',
    'doctopic__topic__title',
    'doctopic__score'
))

[{'content': 'A stator-flux-oriented vector-controlled induction motor drive is described where the space-vector pulsewidth modulation (SVM) and stator-flux-vector estimation are implemented by artificial neural networks (ANNs). ANNs, when implemented by dedicated hardware application-specific integrated circuit chips, provide extreme simplification and fast execution for control and feedback signal processing functions in high-performance ac drives. In the proposed project, a feedforward ANN-based SVM, operating at 20 kHz sampling frequency, generates symmetrical pulsewidth modulation (PWM) pulses in both undermodulation and overmodulation regions covering the range from de (zero frequency) up to square-wave mode at 60 Hz. In addition, a programmable cascaded low-pass filter (PCLPF), that permits dc offset-free stator-flux-vector synthesis at very low frequency using the voltage model, has been implemented by a hybrid neural network which consists of a recurrent neural network (RNN) a

In [189]:
remove_docs = doc_ids - other_t_doc_ids
print(len(remove_docs))
list(Doc.objects.filter(
    pk__in=random.sample(remove_docs,1),
    doctopic__topic__run_id=t.run_id,
    doctopic__score__gt=0.008
).values(
    'title',
    'content',
    'id',
    'doctopic__topic__title',
    'doctopic__score'
))


2298


[{'content': "Recent droughts in Puerto Rico and throughout the Caribbean have emphasized the region's agricultural vulnerability to this hazard and the increasing need for adaptation mechanisms to support sustainable production. In this study, we assessed the geographic extent of agricultural conservation practices incentivized by US Department of Agriculture Natural Resources Conservation Service (NRCS) and evaluated their large-scale contribution to drought adaptability. We identified concentrations of drought-related practices (e.g. cover crops, ponds) applied between 2000 and 2016. Using information from spatial databases and interviews with experts, we assessed the spatial correlation between these practices and areas exposed to drought as identified by the US Drought Monitor. Between 2000 and 2016, Puerto Rico experienced seven drought episodes concentrated around the south, east and southeastern regions. The most profound drought occurred between 2014 and 2016 when the island e

In [190]:
list(Doc.objects.filter(
    pk__in=random.sample(remove_docs,10),
).values(
    'title',#'content'
))

[{'title': 'The chironomid-temperature relationship: expression in nature and palaeoenvironmental implications'},
 {'title': 'Twenty thousand sterling under the sea: Estimating the value of protecting deep-sea biodiversity'},
 {'title': 'Douglas-fir growth in mountain ecosystems: Water limits tree growth from stand to region'},
 {'title': 'Environmental traits affect chironomid communities in glacial areas of the Southern Alps: evidence from a long-lasting case study'},
 {'title': 'Time Scale Analysis and Synthesis for Unmanned Aerial Vehicles(UAVs)'},
 {'title': 'Assessment of cadmium-induced hepatotoxicity and protective effects of zinc against it using an improved cell-based biosensor'},
 {'title': 'Carbon dioxide fixation by Chlorella sp USTB-01 with a fermentor-helical combined photobioreactor'},
 {'title': 'Na+-coupled bicarbonate transporters in duodenum, collecting ducts and choroid plexus'},
 {'title': 'Functional characterization of ACDP2 (ancient conserved domain protein), a

In [191]:
#exclude_docs = Doc.objects.filter(pk__in=remove_docs)

ex_q, created = Query.objects.get_or_create(
    title="Topic Exclusions",
    text="Manually generated",
    creator=User.objects.get(pk=1),
    project=Project.objects.get(pk=186)
)

ex_q.doc_set.clear()

Through = Doc.query.through
dqs = [Through(doc_id=d,query=ex_q) for d in remove_docs]
Through.objects.bulk_create(dqs)

ex_q.r_count = ex_q.doc_set.count()
ex_q.save()

In [192]:
q = Query.objects.get(pk=7360)
ds = set(q.doc_set.values_list('pk',flat=True))
print(len(ds))
drs = ds - remove_docs
print(len(drs))

79475
77177


In [193]:
#exclude_docs = Doc.objects.filter(pk__in=remove_docs)

aex_q, created = Query.objects.get_or_create(
    title="Energy demand all - all docs - topic_exclusions",
    text="Manually generated",
    creator=User.objects.get(pk=1),
    project=Project.objects.get(pk=186)
)
aex_q.doc_set.clear()

Through = Doc.query.through
dqs = [Through(doc_id=d,query=aex_q) for d in drs]
Through.objects.bulk_create(dqs)

aex_q.r_count = aex_q.doc_set.count()
aex_q.save()

In [194]:
q = Query.objects.get(pk=7941)
ds = set(q.doc_set.values_list('pk',flat=True))
print(len(ds))
drs = ds - remove_docs
print(len(drs))

22835
22317


In [195]:
#exclude_docs = Doc.objects.filter(pk__in=remove_docs)

sub_ex_q, created = Query.objects.get_or_create(
    title="No reviews - all docs - topic_exclusions",
    text="Manually generated",
    creator=User.objects.get(pk=1),
    project=Project.objects.get(pk=186)
)
sub_ex_q.doc_set.clear()

Through = Doc.query.through
dqs = [Through(doc_id=d,query=sub_ex_q) for d in drs]
Through.objects.bulk_create(dqs)

sub_ex_q.r_count = sub_ex_q.doc_set.count()
sub_ex_q.save()